<h2 align='center' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

Dataset information: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
Download it from here: https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

#### Import libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
import warnings
warnings.filterwarnings('ignore')

#### Load dataset

In [33]:
df = pd.read_csv("D:\\9.BOOKS MATERIALS\\4.PROGRAMS\\Python_Programs\\Datasets\\sonar_dataset.csv", header=None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
194,0.0392,0.0108,0.0267,0.0257,0.0410,0.0491,0.1053,0.1690,0.2105,0.2471,...,0.0083,0.0080,0.0026,0.0079,0.0042,0.0071,0.0044,0.0022,0.0014,M
157,0.0201,0.0178,0.0274,0.0232,0.0724,0.0833,0.1232,0.1298,0.2085,0.2720,...,0.0131,0.0049,0.0104,0.0102,0.0092,0.0083,0.0020,0.0048,0.0036,M
183,0.0096,0.0404,0.0682,0.0688,0.0887,0.0932,0.0955,0.2140,0.2546,0.2952,...,0.0237,0.0078,0.0144,0.0170,0.0012,0.0109,0.0036,0.0043,0.0018,M
72,0.0208,0.0186,0.0131,0.0211,0.0610,0.0613,0.0612,0.0506,0.0989,0.1093,...,0.0074,0.0063,0.0081,0.0087,0.0044,0.0028,0.0019,0.0049,0.0023,R
161,0.0305,0.0363,0.0214,0.0227,0.0456,0.0665,0.0939,0.0972,0.2535,0.3127,...,0.0200,0.0070,0.0070,0.0086,0.0089,0.0074,0.0042,0.0055,0.0021,M


#### Apply EDA and pre-process the dataset

In [34]:
df.shape

(208, 61)

In [35]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [36]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [37]:
df[60].value_counts() 
# label is not skewed. So, it is a balanced dataset.

M    111
R     97
Name: 60, dtype: int64

In [38]:
# Input features
X = df.drop(60, axis=1)
# Output feature
y = df[60]

In [39]:
y.head()
# contains text data, so do one-hot encoding

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [40]:
y = pd.get_dummies(y, drop_first=True)
y.sample(5) # R --> 1 and M --> 0
# If there are only two dummy columns, then one column can be dropped.

,R
54,1
74,1
43,1
29,1
174,0


In [41]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [42]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


#### Split the dataset into training and testing set

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [44]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


#### Model without Dropout Layer

In [45]:
import tensorflow as tf
from tensorflow import keras

In [46]:
X_train.shape, X_test.shape

((156, 60), (52, 60))

In [47]:
model1 = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6964 - accuracy: 0.5192
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6646 - accuracy: 0.5385
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6527 - accuracy: 0.5705
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6313 - accuracy: 0.6603
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6126 - accuracy: 0.6538
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5816 - accuracy: 0.6923
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5439 - accuracy: 0.7628
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5069 - accuracy: 0.8077
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4779 - accuracy: 0.8077
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4443 - accuracy: 0.8333
Epoch 11/

20/20 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 92/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 93/100


In [48]:
model1.evaluate(X_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.8512 - accuracy: 0.7885


[0.8511648178100586, 0.7884615659713745]

Training Accuracy >>> Test Accuracy

In [49]:
y_pred = model1.predict(X_test).reshape(-1)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[6.5662659e-10 7.6550746e-01 9.9649322e-01 2.4179212e-06 9.9999148e-01
 9.9976820e-01 1.4749373e-02 9.9997216e-01 6.7876231e-06 9.9999666e-01]


In [50]:
# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [51]:
y_test[:10]

,R
186,0
155,0
165,0
200,0
58,1
34,1
151,0
18,1
202,0
62,1


In [52]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.81        27
           1       0.85      0.68      0.76        25

    accuracy                           0.79        52
   macro avg       0.80      0.78      0.78        52
weighted avg       0.80      0.79      0.79        52



#### Model with Dropout Layer

In [53]:
model2 = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7851 - accuracy: 0.5128
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7261 - accuracy: 0.4872
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7256 - accuracy: 0.4872
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7046 - accuracy: 0.4551
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7152 - accuracy: 0.5256
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6969 - accuracy: 0.5064
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7064 - accuracy: 0.5513
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6732 - accuracy: 0.5833
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6832 - accuracy: 0.5513
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6982 - accuracy: 0.5256
Epoch 11/

20/20 [==============================] - 0s 1ms/step - loss: 0.3432 - accuracy: 0.8526
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3480 - accuracy: 0.8590
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3773 - accuracy: 0.8654
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3144 - accuracy: 0.8846
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3236 - accuracy: 0.8462
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3388 - accuracy: 0.8974
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3335 - accuracy: 0.8974
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2958 - accuracy: 0.8910
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3301 - accuracy: 0.8590
Epoch 92/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3894 - accuracy: 0.8526
Epoch 93/100


In [54]:
model2.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.4655 - accuracy: 0.7885


[0.465492844581604, 0.7884615659713745]

Training Accuracy is still good but Test Accuracy Improved

In [55]:
y_pred = model2.predict(X_test).reshape(-1)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[5.8713404e-04 8.6064106e-01 9.1075814e-01 3.6464021e-02 9.9978018e-01
 9.3926525e-01 3.1422853e-01 9.9988753e-01 2.8219674e-02 9.9989629e-01]


In [56]:
# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [57]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.81        27
           1       0.85      0.68      0.76        25

    accuracy                           0.79        52
   macro avg       0.80      0.78      0.78        52
weighted avg       0.80      0.79      0.79        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.81**